In [1]:
#!pip install seaborn
#!pip install openpyxl
#!pip install sklearn
#!pip install yfinance 

In [2]:
import pandas as pd
import numpy as np
import random as rd
import time
import csv
import seaborn as sbs
import yfinance as yf
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [3]:
path_name_results='../results/'
file_result = 'Result_MLP_IBM_stock_prices.csv'

In [4]:
# Define the stock symbol (IBM)
stock_symbol = "IBM"

# Define the start and end dates for the historical data
start_date = "2017-01-01"
end_date = "2023-09-21"

# Download historical data
dataset = yf.download(stock_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1690 entries, 2017-01-03 to 2023-09-20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1690 non-null   float64
 1   High       1690 non-null   float64
 2   Low        1690 non-null   float64
 3   Close      1690 non-null   float64
 4   Adj Close  1690 non-null   float64
 5   Volume     1690 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 92.4 KB


In [6]:
#checks if there are null variables
dataset.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
def salvar_resultado(nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration):
  #Script to write training cycle results
  data = [nm_dataset, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duration]
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['Dataset','Best Params','n_time_steps','MSE','RMSE','MAE','MAPE','sMAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)    

In [8]:
def previsao_MLP(nm_dataset, dataset, n_time_steps):
  # dataframe tratament
  df = pd.DataFrame()
  df['Open']=dataset['Open']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['Open'].shift(n_step)  
      
  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  80% treinam  20% test
  nlinhas = int(np.round(df.shape[0] *0.80)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 1 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 1 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  # Cross-validated for time series
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  param_grid = {
      'hidden_layer_sizes': [(4,6,1),(2,6,1),(6,12,1),(6,18,1)], # MLP layers
      'max_iter': [ 500], # maximum iterations
      'activation': [ 'relu','identity'], # activation function
      'solver': ['adam'], # weight optimization algorithm
      'alpha': [0.0001, 0.001, 0.01],  # alpha strength of regularization
  }
  
  modelo = MLPRegressor(random_state=0)
  
  grid = GridSearchCV(modelo, param_grid, n_jobs= -1,scoring='neg_mean_absolute_percentage_error', cv=ts_cv, verbose=1)
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))  

  # Stores the training execution end time
  Hora_Fim = time.time()  

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio  

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)  
  
  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100   
  
  sMAPE = round(
		np.mean(
			np.abs(predict - y_test) /
			((np.abs(predict) + np.abs(y_test)))
		)*100, 2
	)

  salvar_resultado(nm_dataset, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, sMAPE, Duracao)

In [9]:
#create file to results
criar_arquivo_resultado()

print('forecast for IBM Stock prices')
for n_time_steps in range(1,13): #predict with 1 to 12 past values of medition
    grid = previsao_MLP('IBM', dataset, n_time_steps)

forecast for IBM Stock prices
Fitting 5 folds for each of 24 candidates, totalling 120 fits


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 1, 2.927887814011358, 1.7111071895154195, 1.0372895975331886, 0.9521196656276404, 0.48, 45.17789959907532]
Fitting 5 folds for each of 24 candidates, totalling 120 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 2, 3.2672744517759087, 1.8075603590961793, 1.1307535239928939, 1.0267230456244811, 0.51, 30.62769317626953]
Fitting 5 folds for each of 24 candidates, totalling 120 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 3, 4.378019132463251

c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 4, 5.912792573726397, 2.431623444064972, 1.6147843559189425, 1.4617347799889342, 0.73, 58.02708888053894]
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


...

['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 8, 9.115777537967734, 3.0192345947222674, 2.2193348151497503, 1.8195398460763048, 0.91, 45.41920709609985]
Fitting 5 folds for each of 24 candidates, totalling 120 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (6, 12, 1), 'max_iter': 500, 'solver': 'adam'}", 9, 8.821250868269757, 2.970059068144901, 2.0334770278150955, 1.7882219402495527, 0.89, 28.723921060562134]
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (6, 18, 1), 'max_iter': 500, 'solver': 'adam'}", 10, 6.430877189995292, 2.535917425705201, 1.7460737459760907, 1.5085114552379617, 0.75, 41.685407400131226]
Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (2, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 11, 11.697294184952813, 3.420130726295826, 2.443847738624747, 2.085277200780729, 1.04, 76.67545962333679]
Fitting 5 folds for each of 24 candidates, totalling 120 fits
['Dataset', 'Best Params', 'n_time_steps', 'MSE', 'RMSE', 'MAE', 'MAPE', 'sMAPE', 'Duration']
['IBM', "{'activation': 'identity', 'alpha': 0.01, 'hidden_layer_sizes': (4, 6, 1), 'max_iter': 500, 'solver': 'adam'}", 12, 10.404216479268818, 3.225556770430311, 2.162219348485408, 1.8739753301597668, 0.94, 58.25214958190918]
